# Test MBTR predicting ground state energies of binary systems using KRR 

In [8]:
'''
Authors Brayden Bekker, Chandramouli Nyshadham

Developed as part of the MBTR project using EAM files.

Input:
- [system].eam.alloy: where system is a binary or ternary system of elements from the periodic
table. the potentials were gathered from the ctcms Interatomic Potentials Repository. 
- Structures_[system]/[fcc, bcc, hcp]: A hierarchy of directories for each each system 
containing 2500 derivative super structures developed with enumlib for fcc,bcc,hcp. The first
500 structures and then a random selection of 2000 structures from the 10-12 unit cells.

Output:
- HDF5 with lammps energyies data. See genHDF5 function for details.
'''

#compute total energies with:
import quippy
from lammpslib import LAMMPSlib
from aflow import *
import os
from os.path import isfile, join
import numpy as np
import collections
import matplotlib.pyplot as plt
import h5py
import itertools
import sys
from ase.optimize import BFGS, BFGSLineSearch
from ase.optimize.precon import PreconLBFGS
from tqdm import tqdm

def GetBinarySystem(arg1,arg2):
    '''
    Atomic Simulation Environment (ASE.py) and QUIPPY require information such as the atomic mass, number, etc.. 
    of the elements of a given system to create an atoms object and compute: energy, position, etc...
    
    This function takes as input the elements of the binary system chosen, selects the appropriate data from the 
    dictionary and returns a list of these values for use with quippy, lammps and ase.
    
    Example:
        GetBinarySystem('Al' 'Co')
    Output:
        System: 
    '''

    # The dictionary of all the elements on the periodic table
    elements ={"H":3.75,"AtomicMassH": 1.0079,"AtomicNumH": 1,"He": 3.57,"AtomicMassHe":4.0026,"AtomicNumHe":2, 
               "Li":3.49,"AtomicMassLi":6.941,"AtomicNumLi":3,"Be":2.29,"AtomicMassBe":9.0122,"AtomicNumBe":4,
               "B":8.73,"AtomicMassB":10.811,"AtomicNumB":5,"C": 3.57,"AtomicMassC":12.0107,"AtomicNumC":6, 
               "N":4.039,"AtomicMassN":14.0067,"AtomicNumN":7,"O":6.83,"AtomicMassO":15.9994,"AtomicNumO":8,
               "Ne":4.43,"AtomicMassNe":20.1797,"AtomicNumNe":10,"Na":4.23,"AtomicMassNa":22.9897,"AtomicNumNa":11,
               "Mg":3.21,"AtomicMassMg":24.305,"AtomicNumMg":12,"Al":4.05,"AtomicMassAl":26.9815,"AtomicNumAl":13,
               "Si":5.43,"AtomicMassSi":28.0855,"AtomicNumSi":14,"P":7.17,"AtomicMassP":30.9738,"AtomicNumP":15,
               "S":10.47,"AtomicMassS":32.065,"AtomicNumS":16,"Cl":6.24,"AtomicMassCl":35.453,"AtomicNumCl":17,
               "Ar":5.26,"AtomicMassAr":39.948,"AtomicNumAr":18,"K":5.23,"AtomicMassK":39.0983,"AtomicNumK":19, 
               "Ca":5.58,"AtomicMassCa":40.078,"AtomicNumCa":20,"Sc":3.31,"AtomicMassSc":44.9559,"AtomicNumSc":21,
               "Ti":2.95,"AtomicMassTi":47.867,"AtomicNumTi":22,"V":3.02,"AtomicMassV":50.9415,"AtomicNumV":23, 
               "Cr":2.88,"AtomicMassCr":51.9961,"AtomicNumCr":24,"Mn":8.89,"AtomicMassMn":54.938,"AtomicNumMn":25,
               "Fe":2.87,"AtomicMassFe":55.845,"AtomicNumFe":26,"Co":2.51,"AtomicMassCo":58.9332,"AtomicNumCo":27,
               "Ni":3.52,"AtomicMassNi":58.6934,"AtomicNumNi":28,"Cu":3.61,"AtomicMassCu":63.546,"AtomicNumCu":29,
               "Zn":2.66,"AtomicMassZn":65.39,"AtomicNumZn":30,"Ga":4.51,"AtomicMassGa":69.723,"AtomicNumGa":31,
               "Ge":5.66,"AtomicMassGe":72.64,"AtomicNumGe":32,"As":4.13,"AtomicMassAs":74.9216,"AtomicNumAs":33,
               "Se":4.36,"AtomicMassSe":78.96,"AtomicNumSe":34,"Br":6.67,"AtomicMassBr":79.904,"AtomicNumBr":35, 
               "Kr":5.72,"AtomicMassKr":83.8,"AtomicNumKr":36,"Rb":5.59,"AtomicMassRb":85.4678,"AtomicNumRb":37, 
               "Sr":6.08,"AtomicMassSr":87.62,"AtomicNumSr":38,"Y":3.65,"AtomicMassY":88.9059,"AtomicNumY":39, 
               "Zr":3.23,"AtomicMassZr":91.224,"AtomicNumZr":40,"Nb":3.3,"AtomicMassNb":92.9064,"AtomicNumNb":41,
               "Mo":3.15,"AtomicMassMo":95.94,"AtomicNumMo":42,"Tc":2.74,"AtomicMassTc":98,"AtomicNumTc":43, 
               "Ru":2.7,"AtomicMassRu":101.07,"AtomicNumRu":44,"Rh":3.8,"AtomicMassRh":102.9055,"AtomicNumRh":45,
               "Pd":3.89,"AtomicMassPd":106.42,"AtomicNumPd":46,"Ag":4.09,"AtomicMassAg":107.8682,"AtomicNumAg":47, 
               "Cd":2.98,"AtomicMassCd":112.411,"AtomicNumCd":48,"In":4.59,"AtomicMassIn":114.818,"AtomicNumIn":49, 
               "Sn":5.82,"AtomicMassSn":118.71,"AtomicNumSn":50,"Sb":4.51,"AtomicMassSb":121.76,"AtomicNumSb":51,
               "Te":4.45,"AtomicMassTe":127.6,"AtomicNumTe":52,"I":7.27,"AtomicMassI":126.9045,"AtomicNumI":53, 
               "Xe":6.2,"AtomicMassXe":131.293,"AtomicNumXe":54,"Cs":6.05,"AtomicMassCs":132.9055,"AtomicNumCs":55,
               "Ba":5.02,"AtomicMassBa":137.327,"AtomicNumBa":56,"Hf":3.2,"AtomicMassHf":178.49,"AtomicNumHf":72,
               "Ta":3.31,"AtomicMassTa":180.9479,"AtomicNumTa":73,"W":3.16,"AtomicMassW":183.84,"AtomicNumW":74,
               "Re":2.76,"AtomicMassRe":186.207,"AtomicNumRe":75,"Os":2.64,"AtomicMassOs":190.23,"AtomicNumOs":76,
               "Ir":3.84,"AtomicMassIr":192.217,"AtomicNumIr":77,"Pt":3.92,"AtomicMassPt":195.078,"AtomicNumPt":78,
               "Au":4.08,"AtomicMassAu":196.9665,"AtomicNumAu":79,"Hg":2.99,"AtomicMassHg":200.59,"AtomicNumHg":80,
               "Tl":3.46,"AtomicMassTl":204.3833,"AtomicNumTl":81,"Pb":4.95,"AtomicMassPb":207.2,"AtomicNumPb":82,
               "Bi":4.75,"AtomicMassBi":208.9804,"AtomicNumBi":83,"Sm":3.62,"AtomicMassSm":150.36,"AtomicNumSm":62} 

    element1="%s"%(arg1) #(e.g. Al) Get the element name from the arguments.
    element2="%s"%(arg2)
    mass1="AtomicMass%s"%(arg1) #(e.g. 26.9815) Get the atomic mass of the first element.
    mass2="AtomicMass%s"%(arg2)
    number1="AtomicNum%s"%(arg1) #(e.g. 13) Get the atomic mas of the first element.
    number2="AtomicNum%s"%(arg2)
    #A list of the important atomic information for the given elements.
    system = [element1,element2,elements[mass1],elements[mass2],elements[number1],elements[number2]]
    return system

def readAllVASPFiles(paths,potential,AtomicNum):
    """
    This fuction reads in all the crystal structures in the path to folder given. All the input files should be in
    vasp format. 
    
    The function returns atom numbers, positions(cartesian coordinates), and total energies computed using the
    potential.
    
    Arguments:
        Input: 
            path (string): path to the folder where all strucutres (vasp files) are present
            numStructToRead (int): if FALSE all files in the folder are read,else only 'numStructToRead' are read.
            potential: potential for computing the total energies.
            AtomicNum: Array containing elements atomic numbers.
            
        output:
            Returns 
            
            z (np.array): atomic nuclear charge
            pos (np.array): positions  of all atoms in a crystal structure. (cartesian) 
            lattice (np.array): lattice vectors of the unit cell. 
            totEnePerAtom (np.array): total energy per atom of crystal structure computed using potential.
            structInfo (np.array): contains the structure information (file number).
            conc (np.array): concentration of element A in the binary AB (listed in alphabetical order) 
            forces (np.array): the total force per input concentration.
            
    Note: All files in the folder should be named in the format, 'vasp.1', 'vasp.3', ... etc. which is what enumlib 
            generates.
    """
    
    AtomicNumA=AtomicNum[0]
    AtomicNumB=AtomicNum[1]

    res_z=np.array([])
    res_pos=np.array([])
    res_lattice=np.array([])
    res_totEnePerAtom=np.array([])
    res_structInfo=np.array([])
    res_conc=np.array([])
    res_forces=np.array([])
    

    res={}

    atomsList=quippy.AtomsList()

    for path in paths:

        for i,f  in enumerate(os.listdir(path)):
            inpFile=join(path,f) # get the file name
            a = quippy.Atoms(inpFile, format="vasp") # read the input file as quippy object 
            atomsList.append(a)
            a.set_calculator(potential) # set the potential 
            
            dyn = BFGSLineSearch(a, logfile=None)
            dyn.run(fmax=1e-5)
            print i, f
                
            res_z=np.append(res_z, list(a.z)) # get nuclear charge
            res_pos=np.append(res_pos, a.get_positions()) # get positions in cartesian
            res_lattice=np.append(res_lattice, a.get_cell())  # get lattice vectors

            totalNumOfAtoms=len(a.z) # total number of atoms
            a.params["energy"] = a.get_total_energy()
            res_totEnePerAtom=np.append(res_totEnePerAtom, a.energy/float(len(a.z))) # compute total energy per atom

            res_forces=np.append(res_forces, a.get_total_energy())
            conc=a.z.tolist().count(AtomicNumB)/float(len(a.z)) # get the concentration.
            res_conc=np.append(res_conc, conc)
            

            res_structInfo=np.append(res_structInfo, path[-4::]+str(f)) # structure information a.k.a file name.

    res={'z':res_z,'pos':res_pos,'forces':res_forces, 'lattice':res_lattice,'totEnePerAtom':res_totEnePerAtom,'structInfo':res_structInfo,'conc':res_conc}

    return res #,atomsList

def computeEnthalpy(data):
    """
    
    Input:
    data : Dictionary generated using 
    readAllVASPFiles(paths,potential,numStructToRead=0,AtomicNum=[29,73]) function.'
    
    Output:
    enthalpy added to dictionary 'data'.
    
    """
    
    res_enthalpy=np.array([])
    #find the index of locations where concentration == 0.0;
    index=[i for i,x in enumerate(data['conc']) if x == 0.0] 
    g = np.array([np.where(data['totEnePerAtom'] == np.array(data['totEnePerAtom'])[index].min())])
    fileNum=np.matrix.item(g)
    ene_A=data['totEnePerAtom'][fileNum]
    print "element A", data['structInfo'][fileNum], data['totEnePerAtom'][fileNum]

    index=[i for i,x in enumerate(data['conc']) if x == 1.0]
    g = np.array([np.where(data['totEnePerAtom'] == np.array(data['totEnePerAtom'])[index].min())])
    fileNum=np.matrix.item(g)
    ene_B=data['totEnePerAtom'][fileNum] 
    print "element B", data['structInfo'][fileNum], data['totEnePerAtom'][fileNum]

    for i in range(len(data['conc'])):
        enthalpy = data['totEnePerAtom'][i] - (1-data['conc'][i])*float(ene_A)  - (data['conc'][i])*ene_B
        res_enthalpy=np.append(res_enthalpy, enthalpy)
    
    return res_enthalpy

def plotConvexHull(data,title):
    """
    Plots the convex hull.
    """
    
    fccNum=len(filter(lambda x:'fcc' in x, data['structInfo']))
    bccNum=len(filter(lambda x:'bcc' in x, data['structInfo']))
    hcpNum=len(filter(lambda x:'hcp' in x, data['structInfo']))      
    # plotting the convex hull
    plt.figure()
    plt.scatter(data['conc'][0:fccNum-1],data['enthalpy'][0:fccNum-1],label='fcc')
    plt.scatter(data['conc'][fccNum:fccNum+bccNum-1],data['enthalpy'][fccNum:fccNum+bccNum-1],label='bcc')
    plt.scatter(data['conc'][-hcpNum:],data['enthalpy'][-hcpNum:],label='hcp')
    plt.title(title)
    plt.xlim(0,1)
    plt.ylim(0,5) #increased res of variation at bottom of convex hull.
    plt.legend()
    #plt.savefig("ConvexHull%s.pdf"%(title), format="pdf")
    plt.savefig("ConvexHull2%s.png"%(title), format="png")
    
def get_aflowData(totalStruct,listt):
    """
    Collects AFLOW data for a binary alloy containing species1 and species2
    
    """
    
    resConc=[]
    resEne=[]  
    
    global aflowListt
    
    for i in range(totalStruct):
            
        entry=listt[i]
        resConc.append(entry.stoichiometry[0])
        resEne.append(entry.enthalpy_formation_atom)
        
        
    return resConc,resEne

def plotAflowConvexHull(res1,res2,title):
    # plotting the convex hull.
    plt.figure()
    plt.scatter(res1,res2,s=10,label='alowData')
    plt.xlim(0,1)
    plt.ylim(-1,3)
    plt.legend()
    plt.title(title)
    #plt.savefig("Aflow%s.pdf"%(title), format="pdf") 
    plt.savefig("Aflow2%s.png"%(title), format="png")
    
def getHDF5():
    '''
    create a SystemEnergies.hdf5 file containing information for MBTR:

    The following Hierarchical Structures Exists withing the file with AlTi as an example that
    is repeted for each system in the file.

    SystemEnergies/AlTi/StructInfo: String (e.g. AlTi, fcc, vasp.c)
    SystemEnergies/AlTi/Z: Int (e.g. Nuclear Charges for given vasp.c)
    SystemEnergies/AlTi/BasisVector: Int (e.g. Basis Lattice Vector for given vasp.c)
    SystemEnergies/AlTi/TotAtomEnergy: Int (e.g. Total energy per atom for given vasp.c)
    SystemEnergies/AlTi/TotAtomEnthalpy: Int (e.g. Total enthalpy per atom for given vasp.c)
    SystemEnergies/AlTi/Concentration: Int (e.g. Concentration for given vasp.c)
    '''
    numSystems = 1 #the number of systems to test
    Alsubsystem = ["Co","Ni","Pb","Ti"]
    for i in range(numSystems):
        if(i<8):
            arg3="alloy"
            if(i<4):
                arg1="Al"
                arg2=Alsubsystem[i]
            elif(i==4):
                arg1="Ag"
                arg2="Cu"
            elif(i==5):
                arg1="Co"
                arg2="Ni"
            elif(i==6):
                arg1="Fe"
                arg2="Ni"
            elif(i==7):
                arg1="H"
                arg2="Pd"
        else:
            arg3="fs"
            if(i<11):
                arg1="Al"
                if(i==8): arg2="Fe"
                elif(i==9): arg2="Mg"
                elif(i==10): arg2="Sm"
            elif(i<14):
                arg1="Fe"
                if(i==11): arg2="Mn"
                elif(i==12): arg2="P"
                elif(i==13): arg2="V"
            elif(i<16):
                arg1="Ni"
                if(i==14): arg2="Nb"
                elif(i==15): arg2="Zr"
            elif(i==16):
                arg1="Cu"
                arg2="Zr"
        print arg1, arg2, arg3
        system = GetBinarySystem(arg1, arg2)

        #grp = a.create_group("%s%s"%(system[0],system[1]))

        # loading the interatomic potential.
        header = ["units metal",
                  "dimension 3",
                  "boundary p p p",
                  "atom_style atomic",
                  "atom_modify map array"]
        cmds = ["pair_style eam/%s"%(arg3),
                "pair_coeff * * /root/codes/MBTR_local/eam_potentials/%s-%s.eam.%s %s %s"%(system[0],system[1], arg3, system[0],system[1]),
                "mass 1 %s"%(system[2]),
                "mass 2 %s"%(system[3]),
                "neighbor 2.0 bin",
                "neigh_modify delay 10 check yes"]
        pot = LAMMPSlib(lmpcmds=cmds, atom_types={"%s"%(system[0]): 1, "%s"%(system[1]): 2},
                        lammps_header=header, keep_alive=True)

        pathToDirectories=os.getcwd()+str('/Structures_EAM') #remove MBTR path later
        b="%s%s"%(system[0],system[1])
        folders=[pathToDirectories+str(i) for i in ['/Structures_%s/fcc/'%(b),'/Structures_%s/bcc/'%(b),'/Structures_%s/hcp/'%(b)]]

        data=readAllVASPFiles(folders,potential=pot,AtomicNum=[int(system[4]), int(system[5])])

        #data.update({'enthalpy':computeEnthalpy(data)})

        #plotConvexHull(data,title="%s%s"%(system[0],system[1]))

        #aflowList= search().filter(K.species=="%s,%s"%(system[0],system[1])).filter(K.nspecies==2).orderby(K.enthalpy_formation_atom)

        #totalStruct=len(aflowList)

        #res1,res2=get_aflowData(totalStruct, aflowList)

        #plotAflowConvexHull(res1,res2,"%s%s"%(system[0],system[1]))

        #structSet = grp.create_dataset("StructInfo", data=data["structInfo"])
        #zSet = grp.create_dataset("Z", data=data['z'])
        #posSet = grp.create_dataset("Position", data=data["pos"])
        #vecSet = grp.create_dataset("BasisVector", data=data["lattice"])
        #eneSet = grp.create_dataset("TotAtomEnergy", data=data["totEnePerAtom"])
        #enthSet = grp.create_dataset("TotAtomEnthalpy", data=data["enthalpy"])
        #concSet = grp.create_dataset("Concentration", data=data["conc"])
        #forceSet = grp.create_dataset("Forces", data=data["forces"])
        
getHDF5()

Al Co alloy
                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 14:13:35      -30.452306        6.1646
BFGSLineSearch:    1[  2] 14:13:35      -31.016275        3.5584
BFGSLineSearch:    2[  4] 14:13:35      -31.099808        1.8605
BFGSLineSearch:    3[  6] 14:13:36      -31.136481        1.8234
BFGSLineSearch:    4[  8] 14:13:36      -31.153416        1.1265
BFGSLineSearch:    5[ 10] 14:13:36      -31.162483        0.6960
BFGSLineSearch:    6[ 12] 14:13:36      -31.165698        0.3948
BFGSLineSearch:    7[ 14] 14:13:36      -31.168220        0.3303
BFGSLineSearch:    8[ 16] 14:13:36      -31.169035        0.2041
BFGSLineSearch:    9[ 18] 14:13:36      -31.169148        0.1570
BFGSLineSearch:   10[ 20] 14:13:36      -31.169524        0.1990
BFGSLineSearch:   11[ 22] 14:13:36      -31.169656        0.0763
BFGSLineSearch:   12[ 24] 14:13:36      -31.169733        0.0093
BFGSLineSearch:   13[ 26] 14:13:36      -31.169733        0.0073
BFGSLineSearc

BFGSLineSearch:   83[1742] 14:13:46      -41.803089        0.0000
BFGSLineSearch:   84[1789] 14:13:47      -41.803089        0.0000
BFGSLineSearch:   85[1796] 14:13:47      -41.803089        0.0000
BFGSLineSearch:   86[1845] 14:13:47      -41.803089        0.0000
BFGSLineSearch:   87[1892] 14:13:47      -41.803089        0.0000


KeyboardInterrupt: 

In [ ]:
import h5py
a = h5py.File("SystemEnergies1.hdf5", "w") 

In [ ]:
a.close()

In [5]:
import numpy as np
values = np.arange(1000)
for i in values:
    if
    if(25%i==0):
        print i

0
1
5
25


/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in long_scalars
  after removing the cwd from sys.path.


In [12]:
import os

os.getcwd()
lst=os.listdir("/root/codes/CV/MBTR/Structures_EAM/Structures_AlTi/bcc/")
lst.sort()
print lst

['vasp.1', 'vasp.10', 'vasp.100', 'vasp.101', 'vasp.102', 'vasp.103', 'vasp.104', 'vasp.105', 'vasp.106', 'vasp.107', 'vasp.108', 'vasp.109', 'vasp.11', 'vasp.110', 'vasp.111', 'vasp.112', 'vasp.113', 'vasp.114', 'vasp.115', 'vasp.116', 'vasp.117', 'vasp.118', 'vasp.119', 'vasp.12', 'vasp.120', 'vasp.121', 'vasp.1216', 'vasp.1217', 'vasp.122', 'vasp.1220', 'vasp.123', 'vasp.1234', 'vasp.1235', 'vasp.1237', 'vasp.1238', 'vasp.1239', 'vasp.124', 'vasp.1240', 'vasp.1244', 'vasp.125', 'vasp.1251', 'vasp.1254', 'vasp.1255', 'vasp.1256', 'vasp.1257', 'vasp.126', 'vasp.1260', 'vasp.1266', 'vasp.1267', 'vasp.127', 'vasp.1276', 'vasp.128', 'vasp.1281', 'vasp.1283', 'vasp.1289', 'vasp.129', 'vasp.1290', 'vasp.1296', 'vasp.13', 'vasp.130', 'vasp.1300', 'vasp.1303', 'vasp.1309', 'vasp.131', 'vasp.1314', 'vasp.132', 'vasp.1323', 'vasp.1326', 'vasp.1327', 'vasp.133', 'vasp.1332', 'vasp.1333', 'vasp.1338', 'vasp.134', 'vasp.1345', 'vasp.1347', 'vasp.1348', 'vasp.135', 'vasp.1350', 'vasp.1351', 'vasp.

In [23]:
a={}
b={}
c={}

a={'200': np.arange(100)}
b={'200': np.arange(200)}

a = a['200']
b = b['200']

a = np.append(a, b)

print a

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99   0   1   2   3   4   5   6   7
   8   9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25
  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43
  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61
  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79
  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97
  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115
 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133
 134 135 136 137 138 139 140 141 142 143 144 145 14